In [ ]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from helper.utils import *
from helper.cross_validation import *
from config import *
random_seed_cpu(RANDOM_STATE)

### Load data

In [ ]:
train = pd.read_pickle(join(raw_data_path, "balanced_train.pkl"))
test = pd.read_pickle(join(raw_data_path, "test.pkl"))

In [ ]:
features=train.drop(["target","image_id","event_id"],1).columns

In [ ]:
train["count_>_zero"]=train[features].apply(lambda x :( x>0).sum(),axis=1)
test["count_>_zero"]=test[features].apply(lambda x :( x>0).sum(),axis=1)

train["count_=_zero"]=train[features].apply(lambda x :( x==0).sum(),axis=1)
test["count_=_zero"]=test[features].apply(lambda x :( x==0).sum(),axis=1)

train["mean_>_zero"]=train[features].apply(lambda x :x[x>0].mean(),axis=1)
test["mean_>_zero"]=test[features].apply(lambda x :x[x>0].mean(),axis=1)

train["arg_max"]=train[features].apply(lambda x:int( np.argmax(x).split("_")[-1]),1)
test["arg_max"]=test[features].apply(lambda x:int( np.argmax(x).split("_")[-1]),1)

In [ ]:
features_per_row={ i : [ "feat_"+str(j) for j in range(i*10,(i+1)*10)] 
                 for i in range( 10 )}
features_per_columns ={ i : [ "feat_"+str(j*10+i) for j in range(10)] 
                 for i in range( 10 )}
for key,value in zip(features_per_row.keys(),features_per_row.values()):
    train["count_>_zero_row_"+str(key)]=train[features].apply(lambda x :( x[value]>0).sum(),axis=1)
    test["count_>_zero_row_"+str(key)]=test[features].apply(lambda x :( x[value]>0).sum(),axis=1)
    train["count_=_zero_row_"+str(key)]=train[features].apply(lambda x :( x[value]==0).sum(),axis=1)
    test["count_=_zero_row_"+str(key)]=test[features].apply(lambda x :( x[value]==0).sum(),axis=1)

In [ ]:
for key,value in zip(features_per_columns.keys(),features_per_columns.values()):
    train["count_>_zero_column_"+str(key)]=train[features].apply(lambda x :( x[value]>0).sum(),axis=1)
    test["count_>_zero_column_"+str(key)]=test[features].apply(lambda x :( x[value]>0).sum(),axis=1)
    train["count_=_zero_column_"+str(key)]=train[features].apply(lambda x :( x[value]==0).sum(),axis=1)
    test["count_=_zero_column_"+str(key)]=test[features].apply(lambda x :( x[value]==0).sum(),axis=1)

### Cross validation

In [ ]:
CV = cross_validation(
    train_df=train,
    _id_="image_id",
    target_name="target",
    kfold_type="skfold",
    output_dir=proc_data_path,
    split_ratio=0.1,
    nfolds=5,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=False,
)
train = CV.split()

In [ ]:
version = "v1"
train.to_csv(join(proc_data_path, "train_{}.csv".format(version)), index=False)
test.to_csv(join(proc_data_path, "test_{}.csv".format(version)), index=False)